In [ ]:
#TIER folder

#Original Data
#brain_data - Default to all contrasts
#copy analysis masks into analysis data - done
#need onsets/durations for timeseries variability - done
#Need a txt file with avg motion across first levels - do we also need individual run motion info?
#need behavioral_and_demographic_data - where should this live before TIER?

import os
import os.path as op
from bids import BIDSLayout
import shutil
import glob
import pandas as pd
import copy

In [ ]:
# Get tasks and cons from your BIDS/code/contrasts.tsv file
# Second column is either 'description' or 'desc' in constrasts.tsv
def get_cons_and_tasks(proj_dir, second_column='description'):
    df = pd.read_csv(proj_dir + "data/BIDS/code/contrasts.tsv", sep="\t")
    cons = []
    tasks = []
    for i, row in df.iterrows():
        cons.append("con_%i_%s" % (i+1, row[second_column].lower()))
        if row['task'] not in tasks:
            tasks.append(row['task'])
    return cons, tasks

In [ ]:
###edit this line for your project 
proj_dir = 'CHANGE_THIS_TO_PROJECT_DIRECTORY'

# You can optionally manually input these
tasks = ['read']
cons = ['con_1_tgn-gt-cgn', 
        'con_2_cgd-gt-cgn',
        'con_3_cgf-gt-cgn',
        'con_4_cgf-gt-cgd',
        'con_5_cgd-gt-cgf',
        'con_6_cgn',
        'con_7_cgd',
        'con_8_cgf',
        'con_9_tgn', 
        'con_10_tgn-gt-cgd', 
        'con_11_tgn-gt-cgf']

# Or get them algorithmically
#cons, tasks = get_cons_and_tasks(proj_dir)
track_cons = copy.deepcopy(cons)

print(cons)
print(tasks)

In [ ]:
bids_dir = op.join(proj_dir, 'data', 'BIDS')
layout = BIDSLayout(bids_dir)

#temp_subs = ['sub-{0}'.format(s) for s in layout.get_subjects()]

In [ ]:
#subs = temp_subs
#subs
#subs = temp_subs
subs = [
#e.g. 'sub-SAXEEMOfd01',
 #'sub-SAXEEMOfd02',
 #'sub-SAXEEMOfd03'
 ]

print(subs)

In [ ]:
tier_dir = op.join(proj_dir,'TIER')
anal_dir = op.join(proj_dir, 'Analysis')

#create original data dir, copy zstats, copes, varcopes, and onsets-- copy masks into analysis data dir
copes_dir = op.join(tier_dir,'original_data','copes')
varcopes_dir = op.join(tier_dir,'original_data','varcopes')
zstats_dir = op.join(tier_dir,'original_data','zstats')
onsets_dir = op.join(tier_dir, 'original_data', 'onsets')
contrasts_dir = op.join(tier_dir, 'original_data','contrasts')
mask_dir = op.join(tier_dir,'original_data','masks')

os.makedirs(copes_dir, exist_ok = True)
os.makedirs(varcopes_dir, exist_ok = True)
os.makedirs(zstats_dir, exist_ok = True)
os.makedirs(onsets_dir, exist_ok = True)
os.makedirs(contrasts_dir, exist_ok = True)
os.makedirs(mask_dir, exist_ok = True)

for task_idx, task in enumerate(tasks):
    print(task)

    for sub in subs:
        print(sub)
        
        track_cons = copy.deepcopy(cons)
        subject = op.join(anal_dir, 'second_level_folds', sub)
        if not op.exists(subject):
            print('{} directory not found'.format(subject))
            continue
        subject_dir = op.join(subject,task,'stats/mni')

        for file in os.listdir(subject_dir):
            if not file.startswith('fold'):
                continue
            new_name  = '%s_%s_%s'%(sub,task,op.basename(file))

            if file.endswith("_cope.nii.gz"):
                for con in track_cons:
                    if con in file:
                        track_cons.remove(con)
                        break  
                shutil.copy2(op.join(subject_dir,file),op.join(copes_dir,new_name))
            elif file.endswith("_varcope.nii.gz"):
                shutil.copy2(op.join(subject_dir,file),op.join(varcopes_dir,new_name))
            elif file.endswith("_zstat.nii.gz"):
                shutil.copy2(op.join(subject_dir,file),op.join(zstats_dir,new_name))
            elif file.endswith("_mask.nii.gz"):
                shutil.copy2(op.join(subject_dir,file),op.join(mask_dir,new_name))
        
        # Show if copes are missing
        if track_cons:
            print("Copes that weren't covered for subject " + sub + ":")
            print(*track_cons, sep = "\n")
            print("\n")
                
        onsets = glob.glob(op.join(bids_dir,sub,'func','{}_task-{}_run*events.tsv'.format(sub,task)))
        if onsets:
            for o in onsets:
                shutil.copy2(o, onsets_dir)
                
                
                
#Copy over relevant masks                
mask_dir = op.join(proj_dir, 'data', 'ROI_masks')
tier_mask_dir = op.join(tier_dir,'analysis_data','ROI_MASKS')
os.makedirs(tier_mask_dir, exist_ok = True)

masks = glob.glob(op.join(mask_dir,'*.nii.gz'))

for mask in masks:
    shutil.copy2(mask, tier_mask_dir)

#Copy over contrasts file
contrasts_file = op.join(bids_dir,'code','contrasts.tsv')
shutil.copy2(contrasts_file, op.join(contrasts_dir,'contrasts.tsv'))

In [ ]:
print('done!')